In [49]:
import pandas

results = {
}

for model in ['resnet18', 'resnet34', 'resnet50', 'resnet101']:
    results[model] = dict()
    for mixed in ['no', 'fp16']:
        results[model][mixed] = dict()
        path = f'csv/{model}_{mixed}.csv'
        data = pandas.read_csv(path)
        
        for index, inmodel in enumerate(data['model']):
            splited_data = inmodel.split('_')
            batch_size = splited_data[1]
            num_workers = splited_data[2]
            ga = splited_data[3]

            if batch_size not in list(results[model][mixed].keys()):
                results[model][mixed][batch_size] = dict()
            if num_workers not in list(results[model][mixed][batch_size].keys()):
                results[model][mixed][batch_size][num_workers] = dict()
            if ga not in list(results[model][mixed][batch_size][num_workers].keys()):
                results[model][mixed][batch_size][num_workers][ga] = dict() 

            results[model][mixed][batch_size][num_workers][ga]['cuda_forward_time'] = []
            results[model][mixed][batch_size][num_workers][ga]['cuda_calc_loss_time'] = []
            results[model][mixed][batch_size][num_workers][ga]['cuda_backward_time'] = []
            results[model][mixed][batch_size][num_workers][ga]['cuda_update_time'] = []
            
            if '_32_' in inmodel:
                iters_epoch = 7715
            elif '_64_' in inmodel:
                iters_epoch = 3810
            elif '_128_' in inmodel:
                iters_epoch = 1855
            elif '_256_' in inmodel:
                iters_epoch = 880
            scale = iters_epoch / 1_000
            results[model][mixed][batch_size][num_workers][ga]['cuda_forward_time'].append(list(data['cuda_forward_time'])[index] * scale)
            results[model][mixed][batch_size][num_workers][ga]['cuda_calc_loss_time'].append(list(data['cuda_calc_loss_time'])[index] * scale)
            results[model][mixed][batch_size][num_workers][ga]['cuda_backward_time'].append(list(data['cuda_backward_time'])[index] * scale)
            results[model][mixed][batch_size][num_workers][ga]['cuda_update_time'].append(list(data['cuda_update_time'])[index] * scale)

In [106]:
model = 'resnet18'
mixed = 'no'
time = 'cuda_forward_time'
# model | mixed | batch_size | num_workers | ga | time
print(results[model][mixed]['32']['8']['1'][time])
print(results[model][mixed]['64']['8']['1'][time])
print(results[model][mixed]['128']['8']['1'][time])
print(results[model][mixed]['256']['8']['1'][time])
print()
print()

print(results[model][mixed]['32']['8']['1'][time])
print(results[model][mixed]['64']['8']['1'][time])
print(results[model][mixed]['128']['8']['1'][time])
print(results[model][mixed]['256']['8']['1'][time])
print()

print(results[model][mixed]['32']['8']['2'][time])
print(results[model][mixed]['64']['8']['2'][time])
print(results[model][mixed]['128']['8']['2'][time])
print(results[model][mixed]['256']['8']['2'][time])
print()

print(results[model][mixed]['32']['8']['3'][time])
print(results[model][mixed]['64']['8']['3'][time])
print(results[model][mixed]['128']['8']['3'][time])
print(results[model][mixed]['256']['8']['3'][time])
print()

print(results[model][mixed]['32']['8']['8'][time])
print(results[model][mixed]['64']['8']['8'][time])
print(results[model][mixed]['128']['8']['8'][time])
print(results[model][mixed]['256']['8']['8'][time])
print()

r = results[model]['fp16']['256']['8']['1']

print(r['cuda_forward_time'][0] + r['cuda_calc_loss_time'][0] + r['cuda_backward_time'][0] + r['cuda_update_time'][0])

[62.774052209854126]
[57.311901927471155]
[52.92064630317688]
[50.224377614974976]


[62.774052209854126]
[57.311901927471155]
[52.92064630317688]
[50.224377614974976]

[63.226546812057485]
[57.34744355726242]
[53.02533843421936]
[50.24162372779846]

[63.485773668766015]
[57.44909517908097]
[53.138583959579464]
[50.29065523910523]

[63.52852256917953]
[57.55232306146622]
[53.224043384552004]
[50.29358042144776]

120.4458889716193


In [51]:
import pickle
with open("results.pkl", "wb") as f:
    pickle.dump(results, f)

In [52]:
import numpy as np

batch = list()
label = list()

for model in list(results.keys()):
    for mixed in list(results[model].keys()):
        for batch_size in list(results[model][mixed].keys()):
            for num_workers in list(results[model][mixed][batch_size].keys()):
                for ga in list(results[model][mixed][batch_size][num_workers].keys()):
                    cuda_forward_time = results[model][mixed][batch_size][num_workers][ga]['cuda_forward_time']
                    cuda_calc_loss_time = results[model][mixed][batch_size][num_workers][ga]['cuda_calc_loss_time']
                    cuda_backward_time = results[model][mixed][batch_size][num_workers][ga]['cuda_backward_time']
                    cuda_update_time = results[model][mixed][batch_size][num_workers][ga]['cuda_update_time']
                    
                    total_time = cuda_forward_time[0] + cuda_calc_loss_time[0] + cuda_backward_time[0] + cuda_update_time[0]
                    
                    num_model = int(model.replace("resnet", ""))
                    if mixed == 'no':
                        num_mixed = 32
                    else:
                        num_mixed = 16
                    num_batch_size = int(batch_size)
                    num_num_workers = int(num_workers)
                    num_ga = int(ga)
                    
                    instance = [num_model, num_mixed, num_batch_size, num_num_workers, num_ga]
                    batch.append(instance)
                    label.append(total_time)

np_batch = np.array(batch)
np_label = np.array(label)
print(np_batch.shape, np_label.shape)

np.save('np_batch', np_batch)
np.save('np_label', np_label)

(496, 5) (496,)
